In [1]:
import pandas as pd
import requests
import config
pd.options.display.max_rows = 32

In [2]:
key = config.key # Personal Google Maps API key

In [3]:
subway_stations = pd.read_csv('DOITT_SUBWAY_STATION_01_13SEPT2010.csv')

#Split out, then combine, latitude and longitude
subway_stations['latitude'] = subway_stations['the_geom'].apply(lambda x: x.split()[2].replace(')',''))
subway_stations['longitude'] = subway_stations['the_geom'].apply(lambda x: x.split()[1].replace('(',''))
subway_stations['latitude&longitude'] = subway_stations['latitude']+' '+subway_stations['longitude']

In [4]:
subway_stations.head()

,the_geom,NAME,URL,LINE,latitude,longitude,latitude&longitude
0,POINT (-74.00030814706824 40.73225482650675),W 4th St - Washington Sq (Lower),http://www.mta.info/nyct/subway/index.html,B-D-F-M,40.73225482650675,-74.00030814706824,40.73225482650675 -74.00030814706824
1,POINT (-73.83256899924748 40.846810332614844),Buhre Ave,http://www.mta.info/nyct/subway/index.html,6-6 Express,40.846810332614844,-73.83256899924748,40.846810332614844 -73.83256899924748
2,POINT (-73.97192000013308 40.757107333148234),51st St,http://www.mta.info/nyct/subway/index.html,4-6-6 Express,40.757107333148234,-73.97192000013308,40.757107333148234 -73.97192000013308
3,POINT (-73.97621799811347 40.78864433404891),86th St,http://www.mta.info/nyct/subway/index.html,1-2,40.78864433404891,-73.97621799811347,40.78864433404891 -73.97621799811347
4,POINT (-74.00413100111697 40.713065332984044),Brooklyn Bridge - City Hall,http://www.mta.info/nyct/subway/index.html,4-5-6-6 Express,40.713065332984044,-74.00413100111697,40.713065332984044 -74.00413100111697


In [ ]:
def get_duration_in_minutes(origin, destination, mode='transit', key=key):
    """Return duration in minutes between origin and destination using specified mode of transport.
    Use Google Maps Direction API
    """
    url = 'https://maps.googleapis.com/maps/api/directions/json?origin='+origin+'&destination='+destination+'&mode=transit&transit_mode=rail&key='+key
    # Request directions via public transit
    result = requests.get(url)
    directions = result.json()
    try:
        return directions['routes'][0]['legs'][0]['duration']['value']/60.0
    except:
        print(directions)

def get_durations(df, destinations):
    """Return duration between locations for all rows in dataframe and all destinations"""
    for destination in destinations:
        df[destination] = 0
        
    for origin in df['latitude&longitude']:
        print(origin)
        for destination in destinations:
            print(destination)
            df.loc[df['latitude&longitude']==origin, destination] = get_duration_in_minutes(origin, destination)
            
    return df

def aggregate_durations(df, destinations):
    """Calculate basic statistics about durations in dataframe"""
    df['avg_duration'] = df[destinations].mean(axis=1)
    df['max_duration'] = df[destinations].max(axis=1)
    df['min_duration'] = df[destinations].min(axis=1)
    df['med_duration'] = df[destinations].median(axis=1)
    return df

def replace(df, old, new):
    """Replace durations for old destination with durations for new destination""" 
    df.rename(columns = {old:new}, inplace = True)
    
    for i in range(df['latitude&longitude'].size):
        origin = df['latitude&longitude'][i]
        print(origin)
        destination = new
        url = 'https://maps.googleapis.com/maps/api/directions/json?origin='+origin+'&destination='+destination+'&mode=transit&transit_mode=rail&key='+key
        # Request directions via public transit
        result = requests.get(url)
        directions = result.json()
        df.loc[i, destination] = directions['routes'][0]['legs'][0]['duration']['value']/60.0
    
    return df

destinations = ['Central Park', 'The High Line', '9/11 Memorial', 'Statue of Liberty', \
                'Museum of Modern Art, 11 W 53rd St, New York, NY 10019'.replace(' ', '+'), \
                'Top of the Rock, 30 Rockefeller Plaza, New York, NY 10111'.replace(' ', '+'), \
                'The Frick Collection, 1 E 70th St, New York, NY 10021'.replace(' ', '+'), \
                'Bryant Park', \
                'Grand Central Terminal', 'Times Square', 'Brooklyn Bridge', 'West Village', 'Empire State Building', \
                'New York Transit Museum, 99 Schermerhorn St, Brooklyn, NY 11201'.replace(' ', '+'), 'Brooklyn Botanic Garden', 'Bronx Zoo', \
                'American Museum of Natural History', 'Flushing Main Street', \
                'Whitney Museum of American Art, 99 Gansevoort St, New York, NY 10014'.replace(' ', '+'), \
                'Chelsea Market, 75 9th Ave, New York, NY 10011'.replace(' ', '+'), \
                'The Met Cloisters, 99 Margaret Corbin Dr, New York, NY 10040'.replace(' ', '+')]

In [ ]:
subway_stations = get_durations(subway_stations, destinations)

40.73225482650675 -74.00030814706824
Central Park
The High Line
9/11 Memorial
Statue of Liberty
Museum+of+Modern+Art,+11+W+53rd+St,+New+York,+NY+10019
Top+of+the+Rock,+30+Rockefeller+Plaza,+New+York,+NY+10111
The+Frick+Collection,+1+E+70th+St,+New+York,+NY+10021
Bryant Park
{'status': 'ZERO_RESULTS', 'geocoded_waypoints': [{}, {'geocoder_status': 'OK', 'types': ['establishment', 'park', 'point_of_interest'], 'place_id': 'ChIJ7T_P62dmBzsRqa88VYB0q9w'}], 'available_travel_modes': ['DRIVING', 'WALKING'], 'routes': []}
Grand Central Terminal
Times Square
Brooklyn Bridge
West Village
Empire State Building
New+York+Transit+Museum,+99+Schermerhorn+St,+Brooklyn,+NY+11201
Brooklyn Botanic Garden
Bronx Zoo
American Museum of Natural History
Flushing Main Street
Whitney+Museum+of+American+Art,+99+Gansevoort+St,+New+York,+NY+10014
Chelsea+Market,+75+9th+Ave,+New+York,+NY+10011
The+Met+Cloisters,+99+Margaret+Corbin+Dr,+New+York,+NY+10040
40.846810332614844 -73.83256899924748
Central Park
The High L

In [ ]:
subway_stations = aggregate_durations(subway_stations, destinations)

In [ ]:
subway_stations.sort_values(by='avg_duration', inplace=True)
print(subway_stations[['NAME', 'LINE', 'avg_duration']].head(32))
subway_stations.to_csv('subway_stations.csv', index = False)

                                  NAME             LINE  avg_duration
362  42nd St - Port Authority Bus Term            A-C-E     19.922500
154                 Times Sq - 42nd St      7-7 Express     19.992500
359                 Times Sq - 42nd St            1-2-3     20.297500
437             34th St - Penn Station            A-C-E     20.493333
358             34th St - Penn Station            1-2-3     20.516667
83                 Herald Sq - 34th St            N-Q-R     20.585833
366                Herald Sq - 34th St          B-D-F-M     20.605833
88                  Times Sq - 42nd St            N-Q-R     20.660833
86                  Times Sq - 42nd St                S     21.027500
444                            14th St            A-C-E     21.621667
160                            28th St              1-2     21.636667
87                 42nd St - Bryant Pk          B-D-F-M     21.919167
164                            18th St              1-2     21.965833
441                 

In [ ]:
subway_stations.head()

,the_geom,NAME,URL,LINE,latitude,longitude,latitude&longitude,Central Park,The High Line,9/11 Memorial,...,Bronx Zoo,American Museum of Natural History,Flushing Main Street,"Whitney+Museum+of+American+Art,+99+Gansevoort+St,+New+York,+NY+10014","Chelsea+Market,+75+9th+Ave,+New+York,+NY+10011","The+Met+Cloisters,+99+Margaret+Corbin+Dr,+New+York,+NY+10040",avg_duration,max_duration,min_duration,med_duration
362,POINT (-73.98973500043815 40.75730833180808),42nd St - Port Authority Bus Term,http://www.mta.info/nyct/subway/index.html,A-C-E,40.75730833180808,-73.98973500043815,40.75730833180808 -73.98973500043815,14.450000,11.433333,17.100000,...,51.916667,7.816667,52.900000,12.650000,8.400000,34.666667,19.922500,52.90,7.050000,13.675000
154,POINT (-73.98769099766001 40.75547733499068),Times Sq - 42nd St,http://www.mta.info/nyct/subway/index.html,7-7 Express,40.75547733499068,-73.98769099766001,40.75547733499068 -73.98769099766001,16.916667,12.483333,14.350000,...,47.150000,14.433333,60.900000,14.000000,12.866667,39.133333,19.992500,60.90,6.500000,14.391667
359,POINT (-73.98749500043364 40.75529033316711),Times Sq - 42nd St,http://www.mta.info/nyct/subway/index.html,1-2-3,40.75529033316711,-73.98749500043364,40.75529033316711 -73.98749500043364,19.533333,12.483333,15.350000,...,47.150000,12.300000,58.900000,14.000000,15.583333,39.150000,20.297500,58.90,5.066667,15.466667
437,POINT (-73.99339099930413 40.752287334131175),34th St - Penn Station,http://www.mta.info/nyct/subway/index.html,A-C-E,40.752287334131175,-73.99339099930413,40.752287334131175 -73.99339099930413,16.366667,11.350000,16.016667,...,52.600000,10.733333,42.216667,12.566667,8.316667,38.583333,20.493333,52.60,8.316667,16.191667
358,POINT (-73.99105699933416 40.75037333357704),34th St - Penn Station,http://www.mta.info/nyct/subway/index.html,1-2-3,40.75037333357704,-73.99105699933416,40.75037333357704 -73.99105699933416,20.866667,14.000000,15.050000,...,48.850000,14.233333,43.000000,13.700000,11.816667,41.083333,20.516667,48.85,6.733333,15.083333


In [ ]:
subway_stations[destinations].idxmax(axis=1)

362    Flushing Main Street
154    Flushing Main Street
359    Flushing Main Street
437               Bronx Zoo
358               Bronx Zoo
83                Bronx Zoo
366               Bronx Zoo
88     Flushing Main Street
86     Flushing Main Street
444    Flushing Main Street
160               Bronx Zoo
87     Flushing Main Street
164               Bronx Zoo
441    Flushing Main Street
33     Flushing Main Street
439    Flushing Main Street
               ...         
315    Flushing Main Street
332    Flushing Main Street
336               Bronx Zoo
266    Flushing Main Street
162    Flushing Main Street
331               Bronx Zoo
168    Flushing Main Street
136    Flushing Main Street
135    Flushing Main Street
141    Flushing Main Street
140    Flushing Main Street
142    Flushing Main Street
193               Bronx Zoo
138    Flushing Main Street
137               Bronx Zoo
139               Bronx Zoo
dtype: object

In [ ]:
subway_stations.groupby(by='LINE')['avg_duration'].mean().sort_values()

LINE
B-D-F-M            22.504333
A-C-E              22.525104
1-2-3              22.639028
F-M                22.661250
B-D-E              22.890833
N-Q-R              23.139286
1-2                24.060625
4-5-6-6 Express    25.520694
E-M                26.990833
A-C-F              27.336667
4-5                28.713333
A-B-C              28.718854
A-B-C-D            28.833333
4-6-6 Express      28.987944
2-3                29.559167
7-7 Express-N-W    30.762500
                     ...    
E-J-Z              48.381250
M                  48.855238
2-5                49.349679
3-4                52.951833
J                  54.278690
D                  55.544038
6-6 Express        58.049722
R                  59.750417
A                  60.468155
5                  61.439333
Q                  62.508333
F-Q                63.877500
N                  64.541944
D-F-N-Q            64.587500
A-S                64.861389
2                  67.457500
Name: avg_duration, dtype: float64